Import our Asteroid parser to that we can check our example programs.

In [1]:
from asteroid_interp import interp

# Asteroid: The Language

Asteroid is a general purpose programming language heavily influenced by [Python](https://www.python.org), [Lua](http://www.lua.org), and [ML](https://www.smlnj.org).  The language is based around the main design principles that:

1. Simple things should be simple.
2. Everything is a pattern.

Even though the first design principle seems like common sense try writing a "Hello World" program in [Java](https://java.com/en/).  In Asteroid the canonical "Hello World" program is (programs are written as strings so we can submit them to the Asteroid *interp* function):

In [2]:
hello = \
'''
println "Hello World!".
'''

interp(hello)


(seq 
  |(println 
  |  |(string Hello World!) 
  |  |(file-name 
  |  |  |(nil))) 
  |(nil))


The second design principle makes use of the fact that computation is pattern manipulation. Consider unsigned integer addition:
```C
unsigned short i = 1
unsigned short j = 2
unsigned short k = i + j
```
We think of this as manipulating values but at the machine level this looks like pattern manipulation:
```C
i --> 00000001
j --> 00000010
k --> 00000011
```
The interesting thing about patterns is that we can manipulate them via a process called *pattern matching*.  Asteroid makes this explicit: everything (by everything we mean all expressions) in Asteroid can be interpreted as a pattern/term and therefore manipulated via pattern matching.  

The values derived from patterns are usually customary *interpretations* of the patterns.  Consider the bit patterns above.  In order to derive values from these patterns we interpret the patterns as base two digits.  Other interpretations would of course be possible but perhaps not nearly as convenient.  But the fact remains that it is still just an interpretation with the possibility to change it.

Another example of this kind pattern-value duality is the interpretation of "standard" operators.  In the above example we took it for granted that the `+` operator expresses the integer addition. But this is simply based on the *convention* that certain symbols represent certain operations applied to patterns. Again, patterns and values are attached to each other only by convention with the possibility to change this convention.

Compared to other programming languages Asteroid makes this connection between terms and their interpretations explicit by viewing terms as separate entities from the values that they usually represent.  As we will see, by default Asteroid still provides the "standard" interprestations of the conventional patterns as certain values but the connection is made explicit and can be changed by the user.

Viewing terms and patterns as entities in their own right divorced from their default interpretation opens up the door to an interesting programming paradigm we would like to call *pattern-level* programming where the program has the ability to directly manipulate the terms representing expressions and also has control over the interpretations of those terms.

To get an idea of how Asteroid accomplishes that, here is a small program that changes the standard interpretation of the `+` operator to multiplication.

In [3]:
add_example = \
'''
function funny_add with a, b do
    return a * b.
end function

attach funny_add to __plus__.   -- attach 'funny_add' to '+'
println 3 + 2.                  -- this will print out the value 6
detach from __plus__.           -- restore default interpretation

attach (lambda with a,b do return a*b) to __plus__.   
println 3 + 2.                  -- this will print out the value 6
detach from __plus__.

-- NOTE: '__plus__' is a special symbol representing the '+' operator
'''
interp(add_example)


(seq 
  |(assign 
  |  |(id funny_add) 
  |  |(function 
  |  |  |(body-list 
  |  |  |  |(seq 
  |  |  |  |  |(body 
  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(id a) 
  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |(id b) 
  |  |  |  |  |  |  |  |  |  |(nil))))) 
  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |(times 
  |  |  |  |  |  |  |  |  |  |(id a) 
  |  |  |  |  |  |  |  |  |  |(id b))) 
  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |(nil))))) 
  |(seq 
  |  |(attach 
  |  |  |(fun-id funny_add) 
  |  |  |(constr-id __plus__)) 
  |  |(seq 
  |  |  |(println 
  |  |  |  |(plus 
  |  |  |  |  |(integer 3) 
  |  |  |  |  |(integer 2)) 
  |  |  |  |(file-name 
  |  |  |  |  |(nil))) 
  |  |  |(seq 
  |  |  |  |(detach 
  |  |  |  |  |(id __plus__)) 
  |  |  |  |(seq 
  |  |  |  |  |(attach 
  |  |  |  |  |  |(fun-const 
  |  |

## The Basics

In order to illustrate the basics of Asteroid we turn to the canonical factorial program.

In [4]:
fact = \
'''
-- Factorial

function fact with n do
    if n < 0 do
        error "n has to be non-negative!".
    elif n is 0 do -- this is pattern-level programmng, 'n == 0' is value-level programming.
        return 1.
    else
        return n * fact (n-1).
    end if
end function

println "The factorial of 3 is: ", fact (3).
'''

interp(fact)


(seq 
  |(assign 
  |  |(id fact) 
  |  |(function 
  |  |  |(body-list 
  |  |  |  |(seq 
  |  |  |  |  |(body 
  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |(id n)) 
  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(if 
  |  |  |  |  |  |  |  |  |(cond-exp 
  |  |  |  |  |  |  |  |  |  |(lt 
  |  |  |  |  |  |  |  |  |  |  |(id n) 
  |  |  |  |  |  |  |  |  |  |  |(integer 0))) 
  |  |  |  |  |  |  |  |  |(then-stmt-list 
  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |(error 
  |  |  |  |  |  |  |  |  |  |  |  |(string n has to be non-negative!)) 
  |  |  |  |  |  |  |  |  |  |  |(nil))) 
  |  |  |  |  |  |  |  |  |(elif-list 
  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |(elif 
  |  |  |  |  |  |  |  |  |  |  |  |(cond-exp 
  |  |  |  |  |  |  |  |  |  |  |  |  |(is 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id n) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(integer 0))) 
  |  |  |  |  |  |  |  |  |  | 

Most of the program should be pretty self-explanatory. We have a function definition whose body consists mainly of a cascade of conditional statements.  The computation itself is based on the recursive definition of the factorial operator.  Two things are worth pointing out:

1. The `error` command.  This command throws its argument as an exception term.  In this case the error term is the string appearing in the error command.

2. The condition of the `elif` command.  Here we use the `is` operator to pattern-match the variable `n` against the pattern `0`.  This is the pattern-level approach to programming this condition.  The value-level approach would use the `==` operator comparing the *value* of `n` to the value of the constant `0`.

Here is a program that recursively walks a list and prints out the elements on the list.  It uses the empty list operator `[ ]` and the head-tail operator `[ | ]`  to pattern-match the input list. The `with` and `orwith` blocks allow different patterns to be applied to the input list.  The blocks are tried in the order they appear in the code.  The first one that matches will execute the commands in its code block.  It is an error if none of the blocks match the input to the function.

In [5]:
print_list = \
'''
-- walk a list recursively and print out the elements
function print_list 
    with [] do
        ...     -- empty statements, do nothing!
    orwith [h|t] do
        println h.
        print_list t.
    end function

print_list [1,2,3].
'''

interp(print_list)


(seq 
  |(assign 
  |  |(id print_list) 
  |  |(function 
  |  |  |(body-list 
  |  |  |  |(seq 
  |  |  |  |  |(body 
  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |(nil))) 
  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(noop) 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(noop) 
  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |(noop) 
  |  |  |  |  |  |  |  |  |  |(nil)))))) 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |(| 
  |  |  |  |  |  |  |  |  |  |  |(id h) 
  |  |  |  |  |  |  |  |  |  |  |(id t)) 
  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(println 
  |  |  |  |  |  |  |  |  |  |(id h) 
  |  |  |  |  |  |  |  |  |  |(file-name 
  |  |  |  |  |  |  |  |  |  |  |(nil))) 
  | 

Let's try something a little bit more esoteric.  The following program implements [Peano addition](https://en.wikipedia.org/wiki/Peano_axioms#Addition) over the natural numbers.  For this we need the following definitions:

1. 0 is a natural number.
2. For every natural number n, S(n) is a natural number, where S is the successor function.
3. For all natural numbers m and n, m = n if and only if S(m) = S(n).
4. For every natural number n, S(n) = 0 is false. That is, there is no natural number whose successor is 0.

The following program implements Peano addition using pattern matching on the input term:

In [6]:
peano = \
'''
-- implements Peano addition on terms

-- declare the successor function S as a term constructor so that we 
-- can pattern match on it.

constructor S with arity 1.

-- the 'reduce' function is our reduction engine which recursively pattern matches and
-- rewrites the input term
-- NOTE: during pattern matching free variables are bound to subterms of the original term.
-- For example, the expression S S 0 + 0 is X + 0 will bind X to S S 0 where X was 
-- declared as a variable. Once a pattern value is bound to a variable it can 
-- be used in the program.  In our case we use the values in the variables to 
-- construct new terms, i.e., S reduce (X + Y)

function reduce
    with X + 0 do                      -- pattern match 'X + 0'
            return reduce X.
    orwith X + S Y  do                 -- pattern match to 'X + S Y'
            return S reduce (X + Y).
    orwith term do                     -- default clause
            return term.
    end function

-- construct a term we want to reduce, the quote allows us to construct terms without
-- having Asteroid immediately try to evaluate them.  

let n = 'S S 0 + S S S 0.

-- print the resulting term from our reduction

println reduce n.
'''

interp(peano)


(seq 
  |(assign 
  |  |(id S) 
  |  |(constructor 
  |  |  |(arity 1))) 
  |(seq 
  |  |(assign 
  |  |  |(id reduce) 
  |  |  |(function 
  |  |  |  |(body-list 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |(plus 
  |  |  |  |  |  |  |  |  |(id X) 
  |  |  |  |  |  |  |  |  |(integer 0))) 
  |  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |  |  |  |  |(id reduce) 
  |  |  |  |  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |  |  |  |  |  |(id X) 
  |  |  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |  |(plus 
  |  |  |  |  |  |  |  |  |  |(id X) 
  |  |  |  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |  |  |  |  |(id S) 
  |  |  |  |  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |

Here is another way of doing Peano addition using the built-in 'attach' facility.

In [7]:
peano_attach = \
'''
-- Peano addition using 'attach'

-- declare a constructor for our successor function
constructor S with arity 1.

-- declare a function that implements the behavior for the successor function
function next with n do
    return n + 1.
end function
    
-- attach the behavior/interpretation to the constructor
attach next to S.

-- print value using the attached function
println S S S 0 + S S 0.  
'''
interp(peano_attach)


(seq 
  |(assign 
  |  |(id S) 
  |  |(constructor 
  |  |  |(arity 1))) 
  |(seq 
  |  |(assign 
  |  |  |(id next) 
  |  |  |(function 
  |  |  |  |(body-list 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |(id n)) 
  |  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |  |(plus 
  |  |  |  |  |  |  |  |  |  |  |(id n) 
  |  |  |  |  |  |  |  |  |  |  |(integer 1))) 
  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |(nil))))) 
  |  |(seq 
  |  |  |(attach 
  |  |  |  |(fun-id next) 
  |  |  |  |(constr-id S)) 
  |  |  |(seq 
  |  |  |  |(println 
  |  |  |  |  |(plus 
  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |(id S) 
  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |  |(id S) 
  |  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |  |  |(id S) 
  |  |  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |  |  |  |(integer 0) 
  |  |  |  |  |  |  |  |  |  |(

Here is another way of doing this using function names as constructors.

In [8]:
peano_constr = \
'''
function S with n do
    return n + 1.
end function

println 'S S S 0 + S S 0. -- print term structure using the function name as constructor
println S S S 0 + S S 0. -- print value using the function

'''
interp(peano_constr)


(seq 
  |(assign 
  |  |(id S) 
  |  |(function 
  |  |  |(body-list 
  |  |  |  |(seq 
  |  |  |  |  |(body 
  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |(id n)) 
  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |(plus 
  |  |  |  |  |  |  |  |  |  |(id n) 
  |  |  |  |  |  |  |  |  |  |(integer 1))) 
  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |(nil))))) 
  |(seq 
  |  |(println 
  |  |  |(quote 
  |  |  |  |(plus 
  |  |  |  |  |(juxta 
  |  |  |  |  |  |(id S) 
  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |(id S) 
  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |  |(id S) 
  |  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |  |  |(integer 0) 
  |  |  |  |  |  |  |  |  |(nil))))) 
  |  |  |  |  |(juxta 
  |  |  |  |  |  |(id S) 
  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |(id S) 
  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |  |(integer 0) 
  |  |  |  |  |  |  |  |(nil)))))) 
  |  |  |(file-name 
 

## OO Programming in Asteroid

Asteroid's OO model is heavily influenced by Lua's OO model.  At the core the model is a prototype based model.  The interesting thing of course is that Asteroid supports the full pattern/value duality even in the OO model.  That it, Asteroid allows you to pattern match on instatiated objects.  

In [9]:
hello_OO = \
'''
-- demonstrate our OO model in the context of Asteroid's support for pattern/value duality.

-- declare a named constructor for a new object type
-- the object has two member slots
constructor Hello with arity 2.

-- instantiate a new object using the constructor. members are name-value pairs so that
-- we can use the dictionary-style dot notation to access members.
let hello_obj = Hello
        (
            ("greeting", "Hello World!"),
            ("print_greeting", lambda with self do println self{greeting})
        ).
 
-- use prototyping to instantiate another object of the same kind. 
let another_obj = copy hello_obj.

-- NOTE: in our model of OO we have to pass the object identity to member functions as a 
-- separate parameter. the convention is that it is always the first parameter.

-- the following statements are all equivalent and access the function stored
-- in the second slot.  all these statements print out "Hello World!"
-- in the dot notation the interpreter will automatically generate the object identity pointer
-- in all other cases the user will have to provide it explicitly

hello_obj{print_greeting} none.

hello_obj [1] [1] hello_obj.

with Hello (_, (_,f)) = hello_obj do
    f hello_obj.
end with

-- the following is interesting because it shows that we can pattern match on functions
-- we treat functions are purely syntactic objects -- that is, pattern/value duality is 
-- completely preserved.
let Hello 
    (
        (_, "Hello World!"), 
        (_, lambda with self do println self{greeting})
    )
  = hello_obj.
'''

interp(hello_OO)


(seq 
  |(assign 
  |  |(id Hello) 
  |  |(constructor 
  |  |  |(arity 2))) 
  |(seq 
  |  |(assign 
  |  |  |(id hello_obj) 
  |  |  |(juxta 
  |  |  |  |(id Hello) 
  |  |  |  |(juxta 
  |  |  |  |  |(list 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(string greeting) 
  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |(string Hello World!) 
  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |(string print_greeting) 
  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |(function 
  |  |  |  |  |  |  |  |  |  |  |  |(body-list 
  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id self)) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(stmt-l

Note that in the last line of the example above we diverge drastically from the pattern matching available in the standard functional model.  Consider the following ML program:
```ml
val obj = ("hello world!\n", (fn x => print x));
val ("hello world!\n",f) = obj;
f (#1 obj);   
val ("hello world!\n",(fn x => print x)) = obj;
```
The last line in this code example will fail.  
That means ML is inconsistent in the way it treats objects:
some objects can be inspected using pattern matching and some cannot (functions).
This is due to the fact that function constructors in ML are treated as closures, a  
mixture of syntax and semantics that cannot be viewed as a pattern.
In our approach functions are syntactic objects that can be investigated
using pattern matching making them consistent with other constructors/constants and
preserve our pattern/value duality.

In [10]:
doggies = \
'''
-- another OO example

-- Our Dog type constructor
constructor Dog with arity 3.

-- the prototype object
let dog_proto = Dog (
            ("name", ""),
            ("trick", ""),
            ("make_string", lambda with self do return self{name} + " does " + self{trick})
        ).

-- Fido the dog
let fido = copy dog_proto.
let fido{name} = "Fido".
let fido{trick} = "play dead".
println fido{make_string}().

-- Buddy the dog
let buddy = copy dog_proto.
let buddy{name} = "Buddy".
let buddy{trick} = "roll over".
println buddy{make_string}().
'''
interp(doggies)


(seq 
  |(assign 
  |  |(id Dog) 
  |  |(constructor 
  |  |  |(arity 3))) 
  |(seq 
  |  |(assign 
  |  |  |(id dog_proto) 
  |  |  |(juxta 
  |  |  |  |(id Dog) 
  |  |  |  |(juxta 
  |  |  |  |  |(list 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(string name) 
  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |(string ) 
  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |(string trick) 
  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |(string ) 
  |  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |(string make_string) 
  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |(function 
  |  |  |  |  |  |  |  |  |  |  |  |  |(body-list 
  |

## Sorting

In [11]:
qsort = \
'''
-- Quicksort

function qsort
    with [] do
        return [].
    orwith [a] do
        return [a].
    orwith [pivot|rest] do
        with less=[], more=[] do
            for e in rest do  
                if e < pivot do
                    let less = less + [e].
                else
                    let more = more + [e].
                end if
            end for

            let sorted_less = qsort less.
            let sorted_more = qsort more.

            return sorted_less + [pivot] + sorted_more.
        end with
    end function
'''

interp(qsort)


(seq 
  |(assign 
  |  |(id qsort) 
  |  |(function 
  |  |  |(body-list 
  |  |  |  |(seq 
  |  |  |  |  |(body 
  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |(nil))) 
  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |  |(nil))) 
  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |(id a) 
  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |(id a) 
  |  |  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |  |(p

In [12]:
bubblesort = \
'''
function bubblesort with list do
    with change do
        repeat
            let change = false.
            for i in 0 to length list - 2 do
                if list[i] > list[i+1] do
                    -- swap the values
                    let list[i], list[i+1] = list[i+1], list[i].
                    let change = true.
                end if
            end for
        until not change.
    end with
    return list. -- return the sorted list
end function
'''

interp(bubblesort)


(seq 
  |(assign 
  |  |(id bubblesort) 
  |  |(function 
  |  |  |(body-list 
  |  |  |  |(seq 
  |  |  |  |  |(body 
  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |(id list)) 
  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(with 
  |  |  |  |  |  |  |  |  |(pattern-list 
  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |(assign 
  |  |  |  |  |  |  |  |  |  |  |  |(id change) 
  |  |  |  |  |  |  |  |  |  |  |  |(nil)) 
  |  |  |  |  |  |  |  |  |  |  |(nil))) 
  |  |  |  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |(repeat 
  |  |  |  |  |  |  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(assign 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id change) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(false)) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(for 
  |

  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id change)))) 
  |  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |  |(id list)) 
  |  |  |  |  |  |  |  |  |(nil))))) 
  |  |  |  |  |(nil))))) 
  |(nil))


## Other Random Code Examples

In [13]:
load = \
'''
-- loading modules

load "hello.oid".

println "Jello World!".
'''
interp(load)


(seq 
  |(println 
  |  |(string Hello World!) 
  |  |(file-name 
  |  |  |(nil))) 
  |(seq 
  |  |(println 
  |  |  |(string Jello World!) 
  |  |  |(file-name 
  |  |  |  |(nil))) 
  |  |(nil)))


In [14]:
people =\
'''
-- iterating over compound lists

let people = [("Joe",32,"Cook"),("Peter",24,"Pilot"),("Joanne",45,"Doctor")].

for person in people do
    let name, age, occupation = person. 
    println name + " is " + age + " years old and is a " + occupation. 
end for
'''

interp(people)


(seq 
  |(assign 
  |  |(id people) 
  |  |(list 
  |  |  |(seq 
  |  |  |  |(list 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(string Joe) 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(integer 32) 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(string Cook) 
  |  |  |  |  |  |  |  |(nil))))) 
  |  |  |  |(seq 
  |  |  |  |  |(list 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(string Peter) 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(integer 24) 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(string Pilot) 
  |  |  |  |  |  |  |  |  |(nil))))) 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(string Joanne) 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(integer 45) 
  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |(string Doctor) 
  |  |  |  |  |  |  |  |  |  |(nil))))) 
  |  |  |  |  |  |(nil)))))) 
  |(seq 
  |  |(for 
  |  |  |(id person) 
  |  |  |(in-exp 
  |  |  |  |(id people)) 
  

In [15]:
hello_name =\
'''
-- simple IO
print "Please enter your name: ".
read name.
println "Hello " + name + "!".
'''

interp(hello_name)


(seq 
  |(print 
  |  |(string Please enter your name: ) 
  |  |(file-name 
  |  |  |(nil))) 
  |(seq 
  |  |(read 
  |  |  |(id name) 
  |  |  |(file-name 
  |  |  |  |(nil))) 
  |  |(seq 
  |  |  |(println 
  |  |  |  |(plus 
  |  |  |  |  |(plus 
  |  |  |  |  |  |(string Hello ) 
  |  |  |  |  |  |(id name)) 
  |  |  |  |  |(string !)) 
  |  |  |  |(file-name 
  |  |  |  |  |(nil))) 
  |  |  |(nil))))


In [16]:
tree_walk = \
'''
-- walking a tree defined via tuples of the form (<name>, children*)

let symtab = [
    ("x", 3),
    ("y", 2),
    ("z", 1)
].

function tree_walk
    with ("+", l, r) do
        return tree_walk l + tree_walk r.
    orwith ("-", l, r) do
        return tree_walk l - tree_walk r.
    orwith ("id", name) do
        return symtab{name} otherwise 0. -- choice operator if dictionary returns 'none'
    orwith ("num", i) do
        return i.
    orwith node do
        error "unknown node type", node.
    end function
    
let tree = ("+", ("num", 2), ("id", "x")).

println tree_walk tree.
'''

interp(tree_walk)


(seq 
  |(assign 
  |  |(id symtab) 
  |  |(list 
  |  |  |(seq 
  |  |  |  |(list 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(string x) 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(integer 3) 
  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |(seq 
  |  |  |  |  |(list 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(string y) 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(integer 2) 
  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(string z) 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(integer 1) 
  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |(nil)))))) 
  |(seq 
  |  |(assign 
  |  |  |(id tree_walk) 
  |  |  |(function 
  |  |  |  |(body-list 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |(string +) 
  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  | 

In [17]:
ulisp = \
'''
-- micro lisp interpreter

-- define the type of lisp commands we have
constructor prog with arity 0.
constructor setq with arity 0.
constructor put with arity 0.
constructor id with arity 0.
constructor int with arity 0.
constructor add with arity 0.
constructor sub with arity 0.

-- our symbol table: a list viewed as a dictionary
let symtab = [].

-- our interpreter
function ulisp
    with [] do
        return none.
    orwith [prog|t] do
        with [instr | rest] = t do
            ulisp instr. -- recurse over instruction
            ulisp rest. -- recurse over rest of program
            return none.
        end with
    orwith (setq, v, c) do
        let symtab{v} = ulisp c.  -- dictionary update
        return none.
    orwith (put, c) do
        print ulisp c.
        return none.
    orwith (id, v) do
        return symtab{v} otherwise 0.
    orwith (int, v) do
        return v.
    orwith (add, cl, cr) do
        return ulisp cl + ulisp cr.
    orwith (sub, cl, cr) do
        return ulisp cl - ulisp cr.
    orwith node do
        error "unknown node type", node.
    end function

-- define a program to execute
let p = '(prog,
    (setq, "x", (int, 1)),
    (setq, "y", (int, 2)),
    (put, (add, (id, "x"), (id, "y")))
    ).

-- run it
ulisp p.

'''

interp(ulisp)


(seq 
  |(assign 
  |  |(id prog) 
  |  |(constructor 
  |  |  |(arity 0))) 
  |(seq 
  |  |(assign 
  |  |  |(id setq) 
  |  |  |(constructor 
  |  |  |  |(arity 0))) 
  |  |(seq 
  |  |  |(assign 
  |  |  |  |(id put) 
  |  |  |  |(constructor 
  |  |  |  |  |(arity 0))) 
  |  |  |(seq 
  |  |  |  |(assign 
  |  |  |  |  |(id id) 
  |  |  |  |  |(constructor 
  |  |  |  |  |  |(arity 0))) 
  |  |  |  |(seq 
  |  |  |  |  |(assign 
  |  |  |  |  |  |(id int) 
  |  |  |  |  |  |(constructor 
  |  |  |  |  |  |  |(arity 0))) 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(assign 
  |  |  |  |  |  |  |(id add) 
  |  |  |  |  |  |  |(constructor 
  |  |  |  |  |  |  |  |(arity 0))) 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(assign 
  |  |  |  |  |  |  |  |(id sub) 
  |  |  |  |  |  |  |  |(constructor 
  |  |  |  |  |  |  |  |  |(arity 0))) 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(assign 
  |  |  |  |  |  |  |  |  |(id symtab) 
  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  

  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id int) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id v) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(nil))))) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id v)) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  | 

  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id put) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id add) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id id) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(string x) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |  |  |  |  

In [18]:
interp("with i = 2, j = 2 do println i, j end with")


(seq 
  |(with 
  |  |(pattern-list 
  |  |  |(seq 
  |  |  |  |(assign 
  |  |  |  |  |(id i) 
  |  |  |  |  |(integer 2)) 
  |  |  |  |(seq 
  |  |  |  |  |(assign 
  |  |  |  |  |  |(id j) 
  |  |  |  |  |  |(integer 2)) 
  |  |  |  |  |(nil)))) 
  |  |(stmt-list 
  |  |  |(seq 
  |  |  |  |(println 
  |  |  |  |  |(list 
  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |(id i) 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(id j) 
  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |(file-name 
  |  |  |  |  |  |(nil))) 
  |  |  |  |(nil)))) 
  |(nil))


In [19]:
peano2 = \
'''
-- implements Peano addition using a lookup table for the rewrite rules

-- here we declare the successor function as a term constructor so that we 
-- can pattern match on it.
constructor S with arity 1.

function reduce with term do

    -- rule table uses 'reduce' recursively therefore we need
    -- to declare it here
    let rule_table = 
        [('X + 0, 'reduce X),
         ('X + S Y, 'S reduce (X + Y))].

    -- try to apply the rules to the current term
    for i in 0 to length rule_table - 1 do
        let rule = rule_table[i].
        let lhs, rhs = rule.
        if term is *lhs do
            -- value is a built-in and let's you interpret terms
            -- Note: the default behavior is the evaluation of terms
            -- so we could have just written 'return rhs'
            return value rhs.
        end if
    end for
    return term.

end function

let n = 'S S 0 + S S S 0.

print reduce n.

-- NOTE: quoted expressions are term structures, that is, their structure is validated
-- but they are not evaluated in the current context of the program.  this is important
-- in their use in the rule_table above. here the term, '(S reduce (X + Y))), is validated
-- but not evaluated.  compare this to the expression (S reduce (X + Y))) where the 
-- reduce function would be immediately evaluated without much success because the variables
-- X and Y are free variables and therefore not bound to anything useful.
'''

interp(peano2)


(seq 
  |(assign 
  |  |(id S) 
  |  |(constructor 
  |  |  |(arity 1))) 
  |(seq 
  |  |(assign 
  |  |  |(id reduce) 
  |  |  |(function 
  |  |  |  |(body-list 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |(id term)) 
  |  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(assign 
  |  |  |  |  |  |  |  |  |  |(id rule_table) 
  |  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(quote 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(plus 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id X) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(integer 0))) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(quote 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(juxta 
  |  |  |  |  |  |  |  |  |  |  | 

In [20]:
curried = \
'''
-- Asteroid supports function currying

print (lambda with a do return lambda with b do return a + b) 1 2.
'''

interp(curried)


(seq 
  |(print 
  |  |(juxta 
  |  |  |(function 
  |  |  |  |(body-list 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |(id a)) 
  |  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |  |(function 
  |  |  |  |  |  |  |  |  |  |  |(body-list 
  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |(body 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id b)) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(plus 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id a) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(id b))) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |  |  |  |  |  | 

In [21]:
list_comprehension = \
'''
-- this program builds the matrix
-- [[(1,1), (1,2), (1,3)],
--  [(2,1), (2,2), (2,3)],
--  [(3,1), (3,2), (3,3)]]
-- using list comprehensions. 

let m = [[(j,i) where i in [1,2,3]] where j in [1,2,3]].

-- we could get away with:
let m1 = [(j,i) where i in 1,2,3] where j in 1,2,3.

-- however if we remove the remaining brackets then the program becomes syntactically ambiguous

println m.
'''

interp(list_comprehension)


(seq 
  |(assign 
  |  |(id m) 
  |  |(list 
  |  |  |(seq 
  |  |  |  |(list-where 
  |  |  |  |  |(comp-exp 
  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(list-where 
  |  |  |  |  |  |  |  |  |(comp-exp 
  |  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |(id j) 
  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |(id i) 
  |  |  |  |  |  |  |  |  |  |  |  |  |(nil))))) 
  |  |  |  |  |  |  |  |  |(id i) 
  |  |  |  |  |  |  |  |  |(in-exp 
  |  |  |  |  |  |  |  |  |  |(list 
  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |(integer 1) 
  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |(integer 2) 
  |  |  |  |  |  |  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(integer 3) 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |(nil))))))) 
  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |(id j) 
